In [5]:
import pandas as pd
import numpy as np
from uuid import uuid4
pd.options.display.max_columns = None

In [20]:
master = pd.read_csv('../../data/Cleaned-Jan 9th 22.csv')
# master2['tags'] = master2['Key Story'] + ','+ master2['Looks Marketing'] + ',' + master2['Search Key Word']
# master2['division'] = 'apparel'
# master2['target_audience'] = 'core'
# master2['Gender'] = 'Adult Men'
# master2['Design Exlcusive'] = 'Regular'

/var/folders/gb/1cwn6rds3vv7p30s87jjmp200000gn/T/ipykernel_63350/831162788.py:1: DtypeWarning: Columns (18,20,22,23,24,25,26,27,28,29,30,31,32,34,35,37,40,41,42,43,44,45,46,47,62,67) have mixed types. Specify dtype option on import or set low_memory=False.
  master = pd.read_csv('../../data/Cleaned-Jan 9th 22.csv')


In [7]:
master

,product_id,style_id,brand,gender,division,department,usage,product,sub_product,target_audience,fit,mrp,cost,colour_family,primary_colour,secondary_colour,tertiary_colour,tags,season,exclusive,garment_pattern,print_pattern_type,number_of_components,number_of_pockets,pocket_type,length,neck,collar,placket,sleeve_length,sleeve_type,hemline,waist_rise,closure,footwear_insole,fabric_code,fabric_rate,fabric_story,fabric_composition,fabric_hsn,fabric_weave_pattern,fabric_vendor,denim_cast,denim_wash,wash_care,footwear_upper_material,footwear_sole_material,first_grn_date,first_live_date,first_sold_date,status,created_at,created_by,deleted,ean,myntra_id,ajio_id,ajio_ean,standard_size,barcode_size,garment_waist,inseam_length,to_fit_waist,across_shoulder,chest,front_length,to_fit_foot_length,shoe_weight
0,NaN,TIG-0720-WCT24-01,The Indian Garage Co,Men,Apparel,Ethnicwear,Ethnic,Waistcoat,Waistcoat,Core,Slim,2249,512.0,NaN,Grey,NaN,NaN,"Nehru Jackets,Printed Nehru Jackets,",Winter,Regular,Print,Floral,1,3,Welt Pocket,Short,Round Neck,Mandarin Collar,Concealed Placket,Sleeveless,No Cuff,Asymmetric,NaN,NaN,NaN,Satin-5232,90.0,All Over Prints,Polyester,62032900.0,Satin,Narmada Textiles,NaN,NaN,Hand Wash,NaN,NaN,27-Nov-2020,NaN,NaN,approved,03-01-2023,1,False,8907979160617,13283366.0,4.608258e+11,8.907979e+12,S,S-97CM,NaN,NaN,NaN,15.63,39.5,26.13,NaN,NaN
1,NaN,TIG-0720-WCT24-01,The Indian Garage Co,Men,Apparel,Ethnicwear,Ethnic,Waistcoat,Waistcoat,Core,Slim,2249,512.0,NaN,Grey,NaN,NaN,"Nehru Jackets,Printed Nehru Jackets,",Winter,Regular,Print,Floral,1,3,Welt Pocket,Short,Round Neck,Mandarin Collar,Concealed Placket,Sleeveless,No Cuff,Asymmetric,NaN,NaN,NaN,Satin-5232,90.0,All Over Prints,Polyester,62032900.0,Satin,Narmada Textiles,NaN,NaN,Hand Wash,NaN,NaN,27-Nov-2020,NaN,NaN,approved,03-01-2023,1,False,8907979160624,13283366.0,4.608258e+11,8.907979e+12,M,M-102CM,NaN,NaN,NaN,16.00,41.0,27.50,NaN,NaN
2,NaN,TIG-0720-WCT24-01,The Indian Garage Co,Men,Apparel,Ethnicwear,Ethnic,Waistcoat,Waistcoat,Core,Slim,2249,512.0,NaN,Grey,NaN,NaN,"Nehru Jackets,Printed Nehru Jackets,",Winter,Regular,Print,Floral,1,3,Welt Pocket,Short,Round Neck,Mandarin Collar,Concealed Placket,Sleeveless,No Cuff,Asymmetric,NaN,NaN,NaN,Satin-5232,90.0,All Over Prints,Polyester,62032900.0,Satin,Narmada Textiles,NaN,NaN,Hand Wash,NaN,NaN,27-Nov-2020,NaN,NaN,approved,03-01-2023,1,False,8907979160631,13283366.0,4.608258e+11,8.907979e+12,L,L-107CM,NaN,NaN,NaN,16.38,42.5,27.88,NaN,NaN
3,NaN,TIG-0720-WCT24-01,The Indian Garage Co,Men,Apparel,Ethnicwear,Ethnic,Waistcoat,Waistcoat,Core,Slim,2249,512.0,NaN,Grey,NaN,NaN,"Nehru Jackets,Printed Nehru Jackets,",Winter,Regular,Print,Floral,1,3,Welt Pocket,Short,Round Neck,Mandarin Collar,Concealed Placket,Sleeveless,No Cuff,Asymmetric,NaN,NaN,NaN,Satin-5232,90.0,All Over Prints,Polyester,62032900.0,Satin,Narmada Textiles,NaN,NaN,Hand Wash,NaN,NaN,27-Nov-2020,NaN,NaN,approved,03-01-2023,1,False,8907979160648,13283366.0,4.608258e+11,8.907979e+12,XL,XL-112CM,NaN,NaN,NaN,16.75,44.0,28.25,NaN,NaN
4,NaN,TIG-0720-WCT24-01,The Indian Garage Co,Men,Apparel,Ethnicwear,Ethnic,Waistcoat,Waistcoat,Core,Slim,2249,512.0,NaN,Grey,NaN,NaN,"Nehru Jackets,Printed Nehru Jackets,",Winter,Regular,Print,Floral,1,3,Welt Pocket,Short,Round Neck,Mandarin Collar,Concealed Placket,Sleeveless,No Cuff,Asymmetric,NaN,NaN,NaN,Satin-5232,90.0,All Over Prints,Polyester,62032900.0,Satin,Narmada Textiles,NaN,NaN,Hand Wash,NaN,NaN,27-Nov-2020,NaN,NaN,approved,03-01-2023,1,False,8907979160655,13283366.0,4.608258e+11,8.907979e+12,2XL,2XL-117CM,NaN,NaN,NaN,17.13,45.5,28.63,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49898,NaN,1021-SHOE83-03,The Indian Garage Co,Men,Footwear,Footwear,Chunky Shoes,Sneakers,Sneakers,Core,Regular,2349,595.0,NaN,Wh

In [9]:
# read the file that has colours given by tigc
tigc_colours = pd.read_csv('Colours - tigc colours.csv')
all_colours = set()
all_colours.update(tigc_colours['Barcode Colour'].unique())
all_colours.update(master['primary_colour'].unique())

unique_colours = set()
for colour in all_colours:
    colour = colour.replace('&', '/').split('/')
    if type(colour) == list:
        for c in colour:
            unique_colours.add(c.strip().lower())
    else:
        unique_colours.add(colour.strip().lower())

# export unique colors (they have spelling errors)
pd.DataFrame(list(unique_colours)).to_csv('unique_colors.csv', index=False)

In [11]:
# read the unique colours file with spelling errors fixed
change_color = pd.read_csv('new colors clean - unique_colors.csv')
change_color = change_color[change_color['new'].isnull()==False]
change_color_dict = {}
for index, row in change_color.iterrows():
    change_color_dict[row["old"]] = row["new"]

clean_unique_colours = set()
for colour in unique_colours:
    if change_color_dict.get(colour):
        clean_unique_colours.add(change_color_dict[colour])
    else:
        clean_unique_colours.add(colour)

# export unique & correct spelling colours
pd.DataFrame(list(clean_unique_colours)).to_csv('clean_unique_colors.csv', index=False)

In [12]:
# read the file with unique clean colours and their families
col_fam = pd.read_csv('new colors clean - Sheet1.csv')

In [21]:
# create primary, secondary, colour family columns
for index, row in master.iterrows():
    clean_colours = []
    colours = row['primary_colour'].replace('&', '/').split('/')
    if type(colours) == str:
        colours = [colours]
    for col in colours:
        col = col.strip().lower()
        if change_color_dict.get(col):
            col = change_color_dict[col]
        clean_colours.append(col)
    if len(clean_colours) == 1:
        master.at[index,'primary_colour'] = clean_colours[0]
    elif len(clean_colours) == 2:
        master.at[index,'primary_colour'] = clean_colours[0]
        master.at[index,'secondary_colour'] = clean_colours[1]
    elif len(clean_colours) == 3:
        master.at[index,'primary_colour'] = clean_colours[0]
        master.at[index,'secondary_colour'] = clean_colours[1]
        master.at[index,'tertiary_colour'] = clean_colours[2]
#     print('>>>>', clean_colours[0])
    master.at[index,'colour_family'] = col_fam[col_fam['unique colour'] == clean_colours[0].strip()].values[0][1]

In [78]:
master2[master2['launch Year'] > 2022][['launch Year', 'First GRN']]

,launch Year,First GRN
20596,2023.0,NaN
20597,2023.0,NaN
20598,2023.0,NaN
20599,2024.0,NaN
20600,2023.0,NaN
20601,2025.0,NaN
20950,2023.0,NaN
20951,2024.0,NaN


In [79]:
master2[master2['Ajio Denim Wash'].isnull() == False][['Denim Wash', 'Ajio Denim Wash']].drop_duplicates()

,Denim Wash,Ajio Denim Wash
19390,Ds,Mid Wash
19396,Ss,Light Wash
19801,Rinse,Mid Wash
19807,Ms Rag,Acid Wash
19813,Ms,Mid Wash
20534,Rinse,Clean
20578,DS RAG,Heavy Wash
20590,Ss,Heavy Wash
20677,Ms,Denim
20694,Ds Rag,Heavy Wash


In [80]:
# Department
master2[['Product', 'Sub product', 'Fit']].drop_duplicates().to_csv('prod-sub-fit.csv', index=False)

In [81]:
hie_changes = pd.read_csv('Hierarchy - changes.csv')

product_change = {}
for index, val in hie_changes[hie_changes['column'] == 'Product'].iterrows():
    product_change[val['old']] = val['new']

sub_product_change = {}
for index, val in hie_changes[hie_changes['column'] == 'Sub product'].iterrows():
    sub_product_change[val['old']] = val['new']

fit_change = {}
for index, val in hie_changes[hie_changes['column'] == 'Fit'].iterrows():
    fit_change[val['old']] = val['new']

product_change, sub_product_change, fit_change

({'Jackets (New)': 'Jackets', 'Shirts': 'Shirt'},
 {'5 PKT Chino': '5 pocket Chinos',
  'Cross Pocket': 'Cross Pocket Jeans',
  'Shirts': 'Shirt',
  'Cuffed Cargo': 'Cuffed',
  'Denim Shorts': 'Denim'},
 {'Slim fit': 'Slim Fit'})

In [82]:
hie_clean = pd.read_csv('Hierarchy - clean.csv')
hie_clean
for index, row in master2.iterrows():
    product = row['Product']
    if product_change.get(product):
        product = product_change[product]
        master2.at[index, 'Product'] = product
    sub_product = row['Sub product']
    if sub_product_change.get(sub_product):
        sub_product = sub_product_change[sub_product]
        master2.at[index, 'Sub product'] = sub_product
    fit = row['Fit']
    if fit_change.get(fit):
        fit = fit_change[fit]
        master2.at[index, 'Fit'] = fit
    master2.at[index, 'department'] = hie_clean[(hie_clean['Product'] == product) & (hie_clean['Sub product'] == sub_product) & (hie_clean['Fit'] == fit)]['Department'].values[0]

In [83]:
master2[['Brand', 'Gender', 'division', 'department', 'Usage', 'Product', 'Sub product', 'target_audience','Fit']].drop_duplicates()

,Brand,Gender,division,department,Usage,Product,Sub product,target_audience,Fit
0,The Indian Garage Co,Adult Men,apparel,Jackets and coats,Ethnic,Waistcoat,Waistcoat,core,Slim Fit
15,The Indian Garage Co,Adult Men,apparel,Bottomwear,Casual,Chinos,Chinos,core,Slim Fit
27,The Indian Garage Co,Adult Men,apparel,Bottomwear,Casual,Shorts,Shorts,core,Slim Fit
37,The Indian Garage Co,Adult Men,apparel,Topwear,Casual,Shirt,Shirt,core,Slim Fit
207,The Indian Garage Co,Adult Men,apparel,Denim Bottoms,Casual,Jeans,Jeans,core,Slim Fit
...,...,...,...,...,...,...,...,...,...
20838,The Indian Garage Co,Adult Men,apparel,Bottomwear,Casual,Cargo,Cargo Pants,core,G-BETA FIT CARGO PANT
20898,The Indian Garage Co,Adult Men,apparel,Bottomwear,Casual,Trousers,Elasticated Trousers,core,G-GAMA FIT PANT
20946,The Indian Garage Co,Adult Men,apparel,Denim Bottoms,Casual,Jeans,Jeans,core,G-Ace Slim Fit Jeans
20969,The Indian Garage Co,Adult Men,apparel,Bottomwear,Casual,Shorts,Striped Shorts,core,Slim Fit


In [84]:
# columns dropped from master 2.0:

# Ajio Denim Wash
# launch Year
# SKU
# Country Of Origin
# Manufacturer or Importer Name and Address with Pincode
# Merchandise Exclusive

db_cols = [
    'ean',
    'myntra_id',
    'ajio_id',
    'ajio_ean',
    'product_id',
    'standard_size',
    'barcode_size',
    'style_id',
    'brand',
    'gender',
    'division',
    'department',
    'usage',
    'product',
    'sub_product',
    'target_audience',
    'fit',
    'mrp',
    'cost',
    'colour_family',
    'primary_colour',
    'secondary_colour',
    'tertiary_colour',
    'tags',
    'season',
    'exclusive',
    'garment_pattern',
    'print_pattern_type',
    'number_of_components',
    'number_of_pockets',
    'pocket_type',
    'length',
    'neck',
    'collar',
    'placket',
    'sleeve_length',
    'sleeve_type',
    'hemline',
    'waist_rise',
    'closure',
    'footwear_ankle_type',
    'footwear_insole',
    'fabric_code',
    'fabric_rate',
    'fabric_story',
    'fabric_composition',
    'fabric_hsn',
    'fabric_weave_pattern',
    'fabric_vendor',
    'denim_cast',
    'denim_wash',
    'wash_care',
    'footwear_upper_material',
    'footwear_sole_material',
    'garment_waist',
    'inseam_length',
    'to_fit_waist',
    'across_shoulder',
    'chest',
    'front_length',
    'to_fit_foot_length',
    'shoe_weight',
    'first_grn_date',
    'first_live_date',
    'first_sold_date',
]
final_db = pd.DataFrame(columns=db_cols)

format_map = {
        "ean": "EAN",
        "myntra_id": "myntra Style Id",
        "ajio_id": "Ajio Variant Code",
        "standard_size": "Standard Size",
        "barcode_size": "Barcode Size",
        "style_id": "Style No.",
        "brand": "Brand",
        "gender": "Gender",
        "usage": "Usage",
        "department":"department",
        'division':'division',
        'target_audience': 'target_audience',
        "product": "Product",
        "sub_product": "Sub product",
        "fit": "Fit",
        "mrp": "MRP",
        "cost": "Cost",
        "primary_colour": "primary_colour",
        "secondary_colour": "secondary_colour",
        "colour_family": "colour_family", 
        'tags': 'tags',
        "season": "Season",
        "exclusive": "Design Exlcusive",
        "garment_pattern": "Garment Pattern",
        "print_pattern_type": "Print or Pattern Type",
        "number_of_components": "No Of Components",
        "number_of_pockets": "Number of Pockets",
        "pocket_type": "Pocket Type",
        "length": "Length",
        "neck": "Neck",
        "collar": "Collar",
        "placket": "Placket",
        "sleeve_length": "Sleeve Length",
        "sleeve_type": "Sleeve Type",
        "hemline": "Hemline",
        "waist_rise": "Waist Rise (For Bottomwear)",
        "closure": "Closure (For Bottomwear)",
        "fabric_code": "Fabric Code",
        "fabric_rate": "Fabric Rate",
        "fabric_story": "Fabric Story",
        "fabric_composition": "Fabric Composition",
        "fabric_hsn": "HSN",
        "fabric_weave_pattern": "Fabric Weave Pattern",
        "fabric_vendor": "Vendor Name",
        "denim_cast": "Denim Cast",
        "denim_wash": "Denim Wash",
        "wash_care": "Wash Care",
        "garment_waist": "Garment Waist ( Inches )",
        "inseam_length": "Inseam Length ( Inches )",
        "to_fit_waist": "To Fit Waist ( Inches )",
        "across_shoulder": "Across Shoulder ( Inches )",
        "chest": "Chest ( Inches )",
        "front_length": "Front Length ( Inches )",
        "first_grn_date": "First GRN"   
    }
for mapping in format_map.keys():
    final_db[mapping] = master2[format_map[mapping]]

In [22]:
# Style ids that have more than one set of S,M,L

unique_style_id = {}
issues = set()
for index, row in master2.iterrows():
    if row['style_id'] not in unique_style_id.keys():
        unique_style_id[row['style_id']] = [row['standard_size']]
    else:
        if row['standard_size'] in unique_style_id[row['style_id']]:
            issues.add(row['style_id'])
        unique_style_id[row['style_id']].append(row['standard_size'])
issues

{'0321-SHOE27-01',
 '0421-DNM02-36',
 '0421-SH91-06',
 '0422-HFDNM-180',
 '0422-HFDNM-191',
 '0521-SWT-59-3',
 '0522-CHINO12-01',
 '0522-CHINO12-02',
 '0522-CHINO12-03',
 '0522-CHINO12-04',
 '0522-CHINO12-05',
 '0522-LINTR01-01',
 '0522-LINTR01-02',
 '0522-LINTR01-03',
 '0522-LINTR01-04',
 '0522-LINTR01-05',
 '0522-LINTR01-06',
 '0522-PVCSTR04-01',
 '0522-PVCSTR04-02',
 '0522-PVCSTR04-03',
 '0522-PVCSTR05-01',
 '0522-PVCSTR05-02',
 '0522-PVCSTR05-03',
 '0522-PVCSTR06-01',
 '0522-PVCSTR06-02',
 '0522-PVCSTR07-01',
 '0522-PVCSTR07-02',
 '0522-PVCSTR08-01',
 '0522-PVCSTR08-02',
 '0522-PVCSTR08-03',
 '0522-PVCSTR08-04',
 '0522-PVCSTR08-05',
 '0522-PVCSTR08-06',
 '0522-PVCSTR09-01',
 '0522-PVCSTR09-02',
 '0522-PVCSTR09-03',
 '0522-PVCSTR09-04',
 '0522-PVCSTR09-05',
 '0522-PVCSTR09-06',
 '0522-PVCSTR10-01',
 '0522-PVCSTR10-02',
 '0522-PVCSTR10-03',
 '0522-PVCSTR10-04',
 '0522-PVCSTR10-05',
 '0522-PVCSTR10-06',
 '0622-CHINO01B',
 '0920-CHINO01B',
 '1022- BTCDNM- 17',
 '1022- BTCDNM- 18',
 '11

In [86]:
for style in unique_style_id:
    if style not in issues:
        unique_style_id[style] = str(uuid4())
    else:
        unique_style_id[style] = None
unique_style_id

{'TIG-0720-WCT24-01': 'd24db24b-2d26-4b8a-869d-13d804b580a5',
 'TIG-0720-WCT24-02': 'dba5381b-9deb-467d-9fbc-09c3491ca138',
 'TIG-0720-WCT22-05': 'aee9ef72-34af-4703-b66a-7d5d50d247c1',
 '1119-CHINO04-07': '3f877261-c923-4af7-85a6-056616d7ac0f',
 '1119-CHINO04-06': 'edd58b48-10f2-44c7-82fb-9bc66d9ea196',
 '0920 SHORTS 02-13': '61405d5a-da50-4da8-a3fc-0528540fd462',
 '0920 SHORTS 02-14': '38c6d04e-9f18-4c09-9c45-67c971b218c4',
 '1120-SH57-06': 'a0a9c635-2095-4fc8-b78b-90c772e17ae3',
 '1120-SH57-07': 'fc33d7cc-e763-41e4-b96b-6a8aee4c8bec',
 '1120-SH57-08': '93b06687-551d-4a12-939a-36c0f00d1260',
 '1120-SH57-09': 'a910805e-2315-482d-9ba4-ada86c0e7e4a',
 '1120-SH57-10': 'f5f091f6-722a-435a-abc6-39dc623b1dd6',
 '1120-SH57-11': 'b02ea0ce-7be5-4ba5-812b-655022f16284',
 '1120-SH57-12': 'cccf5421-2a65-4f54-a7f1-cb0645469e8e',
 '1120-SH57-13': 'de9a3ba4-1e99-4798-b620-cdbdd35bb42e',
 '1120-SH57-14': '7dc9443f-76dc-40ee-9a58-6afd7165c452',
 '1120-SH57-15': '32582949-59ac-4625-b336-aac3dd7b23ac',


In [87]:
for index, row in final_db.iterrows():
    final_db.at[index, 'product_id'] = unique_style_id[row['style_id']]

In [88]:
chino_sizes = {}
for index, row in final_db[final_db['style_id'] =='CHINO01B'][['primary_colour', 'standard_size']].iterrows():
    if not chino_sizes.get(row['primary_colour']):
        chino_sizes[row['primary_colour']] = [row['standard_size']]
    else:
        chino_sizes[row['primary_colour']].append(row['standard_size'])
chino_sizes

{'peacot blue': ['28', '30', '32', '34', '36', '38'],
 'brown': ['28', '30', '32', '34', '36', '38'],
 'rust': ['28', '30', '32', '34', '36', '38'],
 'teal blue': ['28', '30', '32', '34', '36', '38'],
 'platinum grey': ['28', '30', '32', '34', '36', '38'],
 'light blue': ['28', '30', '32', '34', '36', '38'],
 'rose': ['28', '30', '32', '34', '36', '38'],
 'pista': ['28', '30', '32', '34', '36', '38'],
 'sky blue': ['28', '30', '32', '34', '36', '38'],
 'tan': ['28', '30', '32', '34', '36', '38'],
 'cloud grey': ['28', '30', '32', '34', '36', '38'],
 'deep water': ['28', '30', '32', '34', '36', '38'],
 'ginger bread': ['28', '30', '32', '34', '36', '38'],
 'harbour blue': ['28', '30', '32', '34', '36', '38'],
 'airforce blue': ['28', '30', '32', '34', '36', '38'],
 'wisteria': ['28', '30', '32', '34', '36', '38'],
 'grape wine': ['28', '30', '32', '34', '36', '38'],
 'silver pink': ['28', '30', '32', '34', '36', '38'],
 'rumba red': ['28', '30', '32', '34', '36', '38']}

In [89]:
for chino in chino_sizes:
    chino_sizes[chino]= str(uuid4())
chino_sizes

{'peacot blue': '89831f31-4085-4c4f-b688-1be930db1b0e',
 'brown': '886606a4-bec7-4895-87f7-14e6b04ea55a',
 'rust': '8a78679f-ea7e-417c-bcbd-038ea030a169',
 'teal blue': '9ffdbe61-b67b-4523-bd61-6fbfc9f0304f',
 'platinum grey': '394bb480-3734-465d-af51-17f1298861af',
 'light blue': '8e17fcf4-2035-4dce-a2cd-f07038459110',
 'rose': '13ae012f-50ca-44a4-abac-d7b040626c6e',
 'pista': '7d81bec8-a301-49b9-9bdb-07bbf77c84a6',
 'sky blue': '6d3f1b23-4e3c-4c21-b14c-63edd39ed256',
 'tan': 'd5f86236-45a2-4a58-931e-d26b5e76f670',
 'cloud grey': 'f7792c23-c73a-4e78-beba-bef1917e3cd3',
 'deep water': 'f2d80b88-e0c5-4d10-9c1c-5fdd70e64f00',
 'ginger bread': 'd3dca606-c9c2-4c4d-bbfc-35ca1bb73353',
 'harbour blue': '799d16d9-a2e9-4478-a2f2-aaced809e8cc',
 'airforce blue': '38f5f108-7dbf-448a-871a-9cd27eeead70',
 'wisteria': 'e00a18d1-005d-4e0e-a940-a62a2b8e7331',
 'grape wine': '35f1d7f5-9d72-4371-b696-418e4ab32edc',
 'silver pink': '043838be-f67b-4872-8bae-3fed7977d14b',
 'rumba red': 'b03b6bff-e3d3-4ba

In [90]:
for index, row in final_db[final_db['style_id'] =='CHINO01B'].iterrows():
    final_db.at[index, 'product_id'] = chino_sizes[row['primary_colour']]

In [91]:
chino_sizes = {}
for index, row in final_db[final_db['style_id'] =='1121-BTEE-41-3'][['primary_colour', 'standard_size']].iterrows():
    if not chino_sizes.get(row['primary_colour']):
        chino_sizes[row['primary_colour']] = [row['standard_size']]
    else:
        chino_sizes[row['primary_colour']].append(row['standard_size'])
for chino in chino_sizes:
    chino_sizes[chino]= str(uuid4())
for index, row in final_db[final_db['style_id'] =='1121-BTEE-41-3'].iterrows():
    final_db.at[index, 'product_id'] = chino_sizes[row['primary_colour']]
chino_sizes = {}
for index, row in final_db[final_db['style_id'] =='0622-CHINO01B'][['primary_colour', 'standard_size']].iterrows():
    if not chino_sizes.get(row['primary_colour']):
        chino_sizes[row['primary_colour']] = [row['standard_size']]
    else:
        chino_sizes[row['primary_colour']].append(row['standard_size'])
for chino in chino_sizes:
    chino_sizes[chino]= str(uuid4())
for index, row in final_db[final_db['style_id'] =='0622-CHINO01B'].iterrows():
    final_db.at[index, 'product_id'] = chino_sizes[row['primary_colour']]

In [94]:
final_db = final_db[final_db['product_id'].isnull() == False]

In [95]:
for index, row in final_db.iterrows():
    if row['standard_size'] == 's':
        final_db.at[index,'standard_size'] = 'S'
    elif row['standard_size'] == '2XL':
        final_db.at[index,'standard_size'] = 'XXL'
    elif row['standard_size'] == '28.00':
        final_db.at[index,'standard_size'] = '28'
    elif row['standard_size'] == '30.00':
        final_db.at[index,'standard_size'] = '30'
    elif row['standard_size'] == '32.00':
        final_db.at[index,'standard_size'] = '32'
    elif row['standard_size'] == '34.00':
        final_db.at[index,'standard_size'] = '34'
    elif row['standard_size'] == '36.00':
        final_db.at[index,'standard_size'] = '36'
    elif row['standard_size'] == '38.00':
        final_db.at[index,'standard_size'] = '38'

In [96]:
fixed_cols = [
    'standard_size',
    'brand',
    'gender',
    'division',
    'department',
    'usage',
    'product',
    'sub_product',
    'target_audience',
    'fit',
    'colour_family',
    'primary_colour',
    'secondary_colour',
    'garment_pattern',
    'print_pattern_type',
    'pocket_type',
    'length',
    'neck',
    'collar',
    'placket',
    'sleeve_length',
    'sleeve_type',
    'hemline',
    'waist_rise',
    'closure',
    'fabric_composition',
    'fabric_weave_pattern',
    'fabric_vendor',
    'denim_cast',
    'denim_wash',
    'wash_care',
]
uniques = {}
for col in fixed_cols:
    uniques[col] = final_db[col].unique().tolist()
uniques

{'standard_size': ['S',
  'M',
  'L',
  'XL',
  'XXL',
  '28',
  '30',
  '32',
  '34',
  '36',
  '38',
  '3XL',
  '40'],
 'brand': ['The Indian Garage Co', 'Freehand', 'Hardsoda'],
 'gender': ['Adult Men'],
 'division': ['apparel'],
 'department': ['Jackets and coats',
  'Bottomwear',
  'Topwear',
  'Denim Bottoms',
  'Loungewear',
  'Knitwear',
  'Denim Top',
  'Winterwear',
  'Sportswear'],
 'usage': ['Ethnic',
  'Casual',
  'Lounge',
  'Street',
  'Athleisure',
  'Resort',
  'Formal',
  'Party',
  'Sports'],
 'product': ['Waistcoat',
  'Chinos',
  'Shorts',
  'Shirt',
  'Jeans',
  'Boxers',
  'Lounge Pants',
  'Lounge Sets',
  'Cargo',
  'Kurta',
  'Joggers',
  'Kurta Pyjama Waistcoat Set',
  'T-Shirt',
  'Jackets',
  'Sweatshirts',
  'Kurta Set',
  'Blazer',
  'Co-ordinates',
  'WindCheater',
  'Knit Tracksuit',
  'Pants',
  'Linen Jogger Trouser',
  'Trousers'],
 'sub_product': ['Waistcoat',
  'Chinos',
  'Shorts',
  'Shirt',
  'Jeans',
  'Regular Boxer',
  'Lounge Pants',
  'Boxe

In [97]:
for index, row in final_db[final_db['number_of_components'] == 'Pack of 2'].iterrows():
    final_db.at[index,'number_of_components'] = 2
for index, row in final_db[final_db['number_of_components'] == '1.00'].iterrows():
    final_db.at[index,'number_of_components'] = 1
for index, row in final_db[final_db['number_of_pockets'] == 'Without Pocket'].iterrows():
    final_db.at[index,'number_of_pockets'] = 0
for index, row in final_db[final_db['number_of_pockets'] == 'Multiple Pockets'].iterrows():
    final_db.at[index,'number_of_pockets'] = -1
for index, row in final_db[final_db['number_of_pockets'].isnull()].iterrows():
    final_db.at[index,'number_of_pockets'] = 0
for index, row in final_db[final_db['number_of_pockets'] == '5.00'].iterrows():
    final_db.at[index,'number_of_pockets'] = 5
for index, row in final_db[final_db['mrp'].isnull()==False].iterrows():
    final_db.at[index,'mrp'] = row['mrp'].replace(',','')
for index, row in final_db[final_db['garment_waist']=='-'].iterrows():
    final_db.at[index,'garment_waist'] = np.nan
for index, row in final_db[final_db['inseam_length']=='-'].iterrows():
    final_db.at[index,'inseam_length'] = np.nan
for index, row in final_db[final_db['to_fit_waist']=='-'].iterrows():
    final_db.at[index,'to_fit_waist'] = np.nan
for index, row in final_db[final_db['across_shoulder']=='-'].iterrows():
    final_db.at[index,'across_shoulder'] = np.nan
for index, row in final_db[final_db['chest']=='-'].iterrows():
    final_db.at[index,'chest'] = np.nan
for index, row in final_db[final_db['front_length']=='-'].iterrows():
    final_db.at[index,'front_length'] = np.nan

In [98]:
for col in ['across_shoulder', 'chest','front_length']:
    final_db[col] = final_db[col].str.replace(" 1/2", ".5")
    final_db[col] = final_db[col].str.replace(" 1/4", ".25")
    final_db[col] = final_db[col].str.replace(" 3/4", ".75")
    final_db[col] = final_db[col].str.replace(" 3/8", ".375")
    final_db[col] = final_db[col].str.replace(" 5/8", ".625")
    final_db[col] = final_db[col].str.replace(" 7/8", ".875")

In [99]:
for index, row in final_db.iterrows():
    try:
        final_db.at[index,'fabric_rate'] = float(row['fabric_rate'])
    except:
        final_db.at[index,'fabric_rate'] = np.nan

In [100]:
final_db['ean'] = final_db['ean'].astype(int)
final_db['fabric_hsn'] = pd.to_numeric(final_db['fabric_hsn'], errors='coerce').fillna(0).astype(int)
final_db['ajio_id'] = pd.to_numeric(final_db['ajio_id'], errors='coerce').fillna(0).astype(int)
final_db['myntra_id'] = pd.to_numeric(final_db['myntra_id'], errors='coerce').fillna(0).astype(int)

final_db['number_of_components'] = final_db['number_of_components'].astype(int)
final_db['number_of_pockets'] = final_db['number_of_pockets'].astype(int)

final_db['mrp'] = final_db['mrp'].astype(float)
final_db['cost'] = final_db['cost'].astype(float)
final_db['fabric_rate'] = final_db['fabric_rate'].astype(float)
final_db['garment_waist'] = final_db['garment_waist'].astype(float)
final_db['inseam_length'] = final_db['inseam_length'].astype(float)
final_db['to_fit_waist'] = final_db['to_fit_waist'].astype(float)
final_db['across_shoulder'] = final_db['across_shoulder'].astype(float)
final_db['chest'] = final_db['chest'].astype(float)
final_db['front_length'] = final_db['front_length'].astype(float)

In [101]:
# final_db['fabric_hsn'].replace(0,np.nan, inplace=True)
# final_db['ajio_id'].replace(0,np.nan, inplace=True)
# final_db['myntra_id'].replace(0,np.nan, inplace=True)

In [102]:
{'footwear_ankle_type' : final_db['footwear_ankle_type'].unique(),
'footwear_insole' : final_db['footwear_insole'].unique(),
'footwear_upper_material' : final_db['footwear_upper_material'].unique(),
'footwear_sole_material' : final_db['footwear_sole_material'].unique(),
'to_fit_foot_length' : final_db['to_fit_foot_length'].unique(),
'shoe_weight' : final_db['shoe_weight'].unique(),
'ajio_ean' : final_db['ajio_ean'].unique(),
'tertiary_colour' : final_db['tertiary_colour'].unique()}

{'footwear_ankle_type': array([nan], dtype=object),
 'footwear_insole': array([nan], dtype=object),
 'footwear_upper_material': array([nan], dtype=object),
 'footwear_sole_material': array([nan], dtype=object),
 'to_fit_foot_length': array([nan], dtype=object),
 'shoe_weight': array([nan], dtype=object),
 'ajio_ean': array([nan], dtype=object),
 'tertiary_colour': array([nan], dtype=object)}

In [103]:
final_db['first_grn_date'] = pd.to_datetime(final_db['first_grn_date'], format='%d-%b-%Y')

In [104]:
final_db.to_csv('clean_db.csv',index=False)

In [105]:
sizes = []
styles = []
unique_products = set()

for index, row in final_db.iterrows():
    if row['product_id'] not in unique_products:
        unique_products.add(row['product_id'])
        styles.append({
            'product_id': row['product_id'],
            'style_id': row['style_id'],
            'brand': row['brand'],
            'gender': row['gender'],
            'division': row['division'],
            'department': row['department'],
            'usage': row['usage'],
            'product': row['product'],
            'sub_product': row['sub_product'],
            'target_audience': row['target_audience'],
            'fit': row['fit'],
            'mrp': row['mrp'],
            'cost': row['cost'],
            'colour_family': row['colour_family'],
            'primary_colour': row['primary_colour'],
            'secondary_colour': row['secondary_colour'],
            'tertiary_colour': row['tertiary_colour'],
            'tags': row['tags'],
            'season': row['season'],
            'exclusive': row['exclusive'],
            'garment_pattern': row['garment_pattern'],
            'print_pattern_type': row['print_pattern_type'],
            'number_of_components': row['number_of_components'],
            'number_of_pockets': row['number_of_pockets'],
            'pocket_type': row['pocket_type'],
            'length': row['length'],
            'neck': row['neck'],
            'collar': row['collar'],
            'placket': row['placket'],
            'sleeve_length': row['sleeve_length'],
            'sleeve_type': row['sleeve_type'],
            'hemline': row['hemline'],
            'waist_rise': row['waist_rise'],
            'closure': row['closure'],
            'footwear_ankle_type': row['footwear_ankle_type'],
            'footwear_insole': row['footwear_insole'],
            'fabric_code': row['fabric_code'],
            'fabric_rate': row['fabric_rate'],
            'fabric_story': row['fabric_story'],
            'fabric_composition': row['fabric_composition'],
            'fabric_hsn': row['fabric_hsn'],
            'fabric_weave_pattern': row['fabric_weave_pattern'],
            'fabric_vendor': row['fabric_vendor'],
            'denim_cast': row['denim_cast'],
            'denim_wash': row['denim_wash'],
            'wash_care': row['wash_care'],
            'footwear_upper_material': row['footwear_upper_material'],
            'footwear_sole_material': row['footwear_sole_material'],
            'first_grn_date': row['first_grn_date'],
            'first_live_date': row['first_live_date'],
            'first_sold_date': row['first_sold_date'],
        })
    sizes.append({
        'ean':row['ean'],
        'myntra_id':row['myntra_id'],
        'ajio_id':row['ajio_id'],
        'ajio_ean':row['ajio_ean'],
        'product_id':row['product_id'],
        'standard_size':row['standard_size'],
        'barcode_size':row['barcode_size'],
        'garment_waist':row['garment_waist'],
        'inseam_length':row['inseam_length'],
        'to_fit_waist':row['to_fit_waist'],
        'across_shoulder':row['across_shoulder'],
        'chest':row['chest'],
        'front_length':row['front_length'],
        'to_fit_foot_length':row['to_fit_foot_length'],
        'shoe_weight':row['shoe_weight'],
    })

In [106]:
styles = pd.DataFrame.from_records(styles)
sizes = pd.DataFrame.from_records(sizes)

In [110]:
styles['status'] = 'approved'
styles['created_at'] = '2022-11-13'
styles['created_by'] = 1
styles['deleted'] = False
styles['fabric_rate'].replace(np.nan, 0, inplace=True)
styles['fabric_vendor'].replace(np.nan, 'unknown', inplace=True)
styles['fabric_code'].replace(np.nan, 'unknown', inplace=True)

In [111]:
styles.to_csv('styles.csv', index=False)
sizes.to_csv('sizes.csv', index=False)

In [93]:
# not clean styles
# '0421-DNM02-36'
# '0421-SH91-06'
# '0422-HFDNM-180'
# '0422-HFDNM-191'
# '0521-SWT-59-3'

In [18]:
db = pd.read_csv('clean_db.csv')
options = {}
for col in ['standard_size',
'brand',
'gender',
'division',
'department',
'usage',
'product',
'sub_product',
'target_audience',
'fit',
'colour_family',
'primary_colour',
'exclusive',
'garment_pattern',
'print_pattern_type',
'number_of_components',
'number_of_pockets',
'pocket_type',
'length',
'neck',
'collar',
'placket',
'sleeve_length',
'sleeve_type',
'hemline',
'waist_rise',
'closure',
'footwear_ankle_type',
'footwear_insole',
'fabric_story',
'fabric_composition',
'fabric_weave_pattern',
'fabric_vendor',
'denim_cast',
'denim_wash',
'wash_care',
'footwear_upper_material',
'footwear_sole_material']:
    all_options = db[col].unique().tolist()
    options[col] = all_options
import json

# remove nan manually
with open("allowed_options.json", "w") as outfile:
    json.dump(options, outfile)

/var/folders/gb/1cwn6rds3vv7p30s87jjmp200000gn/T/ipykernel_4948/3828665120.py:1: DtypeWarning: Columns (23,49,62) have mixed types. Specify dtype option on import or set low_memory=False.
  db = pd.read_csv('clean_db.csv')
